In [1]:
import pandas as pd

## Load tables

In [2]:

try: 
    mq = pd.read_csv('mq.csv')
    nftfi = pd.read_csv('nftfi.csv')
except FileNotFoundError:
    mq = pd.read_csv('analysis/mq.csv')
    nftfi = pd.read_csv('analysis/nftfi.csv')

In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)


In [4]:
print('mq')
display(mq.head(2))
print('--------------------------------------------------------------------')
print('nftfi')
display(nftfi.head(2))

mq


transaction_hash  \
0  0x933ff7d3b9d60593c97a5c9ab28205e54f019259defd...   
1  0xd1b9fbddbcaaaea8fa8b19cfa53c5e9093993f0d51c0...   

                  block_timestamp  loan_id  \
0  2022-02-26 09:26:37.000000 UTC    -4882   
1  2022-04-19 22:33:16.000000 UTC      733   

                                   to_address  \
0  0x676f3d2c9f5776e09d23986adc6684fb41e9000e   
1  0x676f3d2c9f5776e09d23986adc6684fb41e9000e   

                                 from_address  principal_amount  \
0  0x0aff497bd016000185b1c8302fa98a88ff4a4178               0.8   
1  0x2c35b3d207f745ad5b17d88328286c49a0ce4d4f               1.4   

   repayment_amount                               erc20_address erc20_name  \
0          0.826624  0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2       wETH   
1          1.413425  0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2       wETH   

                         due_date  duration_in_days       apr token_id  \
0  2022-03-28 09:26:37.000000 UTC              30.0  0.405000     1378   
1  2022-04-26 22:33:16.000000 UTC               7.0  0.500013     1378   

                           collection_address protocol  amt_in_usd  roll_over  \
0  0x026224a2940bfe258d0dbe947919b62fe321f042    nftfi  2205.66016      False   
1  0x026224a2940bfe258d0dbe947919b62fe321f042    nftfi  4339.42642      False   

   block_number p2p_p2pool  
0      14280871        p2p  
1      14618284        p2p

--------------------------------------------------------------------
nftfi


date blockchain collection_name  \
0  2022-03-30 10:43:05.000000 UTC   ethereum      Art Blocks   
1  2022-07-04 17:30:26.000000 UTC   ethereum      Art Blocks   

              loan_no                 loan_start_time  \
0     v2.loan.fixed-3  2022-03-30 10:43:05.000000 UTC   
1  v2.loan.fixed-6396  2022-07-04 17:30:26.000000 UTC   

                    loan_due_time  repaid  no_of_days  liquidated  \
0  2022-03-30 10:50:17.000000 UTC    True       0.005        True   
1  2022-08-04 17:30:26.000000 UTC    True      31.000       False   

   loan_principal_amount  maximum_repayment_amount  eth_price   usd_value  \
0                   0.01              1.000010e-20    3398.59     33.9859   
1                   9.00              9.152877e-18    1127.23  10145.0700   

       interest                                      lender  \
0  1.000000e-07  0xd79275564CCAd8E679cbBb5E1C2FaCD46643a372   
1  1.528770e-01  0xE0450df8d87C94B52e779e8674746EBfa7e6D04B   

                                     borrower  \
0  0x3e3B47b5d433bE4708A6BD524faCd48Bb54D10b1   
1  0x8264e9e0f4CbcBBbb3F8eCAec0A625B590Ae790e   

                      nft_collateral_contract  nft_collateral_id  active  \
0  0x059edd72cd353df5106d2b9cc5ab83a52287ac3a            1000323   False   
1  0x059edd72cd353df5106d2b9cc5ab83a52287ac3a               5510   False   

         apr                      loan_erc20denomination  \
0  73.000000  0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2   
1  20.000038  0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2   

                 loan_repaid_time           loan_liquidation_time  
0                             NaN  2022-03-30 10:59:51.000000 UTC  
1  2022-08-04 16:01:50.000000 UTC                             NaN

In [5]:
mq['loan_id'].head(2)

0   -4882
1     733
Name: loan_id, dtype: int64

### Clean data

In [6]:
# clean loan IDs to try and compare them
mq['loan_id_cleaned'] = mq['loan_id'].astype(str).str.replace('-','')
nftfi['loan_no_cleaned'] = nftfi['loan_no'].str.split('-').apply(lambda x: x[-1])

In [7]:
display(mq['loan_id_cleaned'])
display(nftfi['loan_no_cleaned'])

0         4882
1          733
2        31180
3        27589
4        23358
         ...  
22037    20293
22038    20292
22039    20291
22040    20290
22041    20288
Name: loan_id_cleaned, Length: 22042, dtype: object

0            3
1         6396
2         7075
3         7722
4        12739
         ...  
19494    18848
19495    18854
19496    18876
19497    19124
19498    19125
Name: loan_no_cleaned, Length: 19499, dtype: object

In [8]:
mq.shape[0], nftfi.shape[0]

(22042, 19499)

In [9]:
# Check minimum and maximum dates of each table
print(f"min mq date: {mq.block_timestamp.min()}; max mq date: {mq.block_timestamp.max()}")
print(f"min nftfi date: {nftfi.date.min()}; max nftfi date: {nftfi.date.max()}")

min mq date: 2020-10-15 18:15:24.000000 UTC; max mq date: 2023-05-18 12:41:47.000000 UTC
min nftfi date: 2022-03-30 10:21:32.000000 UTC; max nftfi date: 2023-05-15 10:29:23.000000 UTC


## Compute April subset for each table

In [10]:
april_mq = mq.loc[mq['block_timestamp'] >= '2023-04-01'].loc[mq['block_timestamp'] < '2023-05-01']
april_mq.shape

(1852, 20)

In [11]:
april_nftfi = nftfi.loc[nftfi['date'] >= '2023-04-01'].loc[nftfi['date'] < '2023-05-01']
april_nftfi = april_nftfi[~(april_nftfi['blockchain'] == 'optimism')]
april_nftfi.shape

(2168, 24)

#### Check for duplicates

In [12]:
april_nftfi.loc[april_nftfi.loan_no_cleaned.duplicated()== True]

date blockchain collection_name  \
2835   2023-04-10 20:33:35.000000 UTC   ethereum   Kanpai Pandas   
10030  2023-04-27 00:15:11.000000 UTC   ethereum   Kanpai Pandas   

                              loan_no                 loan_start_time  \
2835   v2.loan.fixed.collection-32241  2023-04-10 20:33:35.000000 UTC   
10030           v2-1.loan.fixed-34175  2023-04-27 00:15:11.000000 UTC   

                        loan_due_time  repaid  no_of_days  liquidated  \
2835   2023-04-13 20:33:35.000000 UTC    True         3.0       False   
10030  2023-05-11 00:15:11.000000 UTC    True        14.0       False   

       loan_principal_amount  maximum_repayment_amount  eth_price  \
2835            1.281000e+00              1.290476e-18    1883.01   
10030           1.700000e-09              1.730000e-27    1894.89   

          usd_value      interest                                      lender  \
2835   2.412136e+03  9.475890e-03  0x2ae57B80DF1Ea04E5638E780ecc6c96232be65D5   
10030  1.700000e-09  3.000000e-11  0x226d3154Ea70DCC8b983f4BD4061D8BED318E03f   

                                         borrower  \
2835   0x331739Ad658F76dAbc0F5790d42599Ac38DaAD91   
10030  0x331739Ad658F76dAbc0F5790d42599Ac38DaAD91   

                          nft_collateral_contract  nft_collateral_id  active  \
2835   0xacf63e56fd08970b43401492a02f6f38b6635c91               4696   False   
10030  0xacf63e56fd08970b43401492a02f6f38b6635c91               4695   False   

             apr                      loan_erc20denomination  \
2835   90.000000  0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2   
10030  46.008403  0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48   

                     loan_repaid_time loan_liquidation_time loan_no_cleaned  
2835   2023-04-12 13:40:11.000000 UTC                   NaN           32241  
10030  2023-05-06 19:03:23.000000 UTC                   NaN           34175

In [13]:
april_mq.loc[april_mq.loan_id_cleaned.duplicated()== True]

Empty DataFrame
Columns: [transaction_hash, block_timestamp, loan_id, to_address, from_address, principal_amount, repayment_amount, erc20_address, erc20_name, due_date, duration_in_days, apr, token_id, collection_address, protocol, amt_in_usd, roll_over, block_number, p2p_p2pool, loan_id_cleaned]
Index: []

In [14]:
april_nftfi = april_nftfi[~(april_nftfi['blockchain'] == 'optimism')]

In [15]:
april_nftfi.loc[april_nftfi.loan_no_cleaned.duplicated()== True]

date blockchain collection_name  \
2835   2023-04-10 20:33:35.000000 UTC   ethereum   Kanpai Pandas   
10030  2023-04-27 00:15:11.000000 UTC   ethereum   Kanpai Pandas   

                              loan_no                 loan_start_time  \
2835   v2.loan.fixed.collection-32241  2023-04-10 20:33:35.000000 UTC   
10030           v2-1.loan.fixed-34175  2023-04-27 00:15:11.000000 UTC   

                        loan_due_time  repaid  no_of_days  liquidated  \
2835   2023-04-13 20:33:35.000000 UTC    True         3.0       False   
10030  2023-05-11 00:15:11.000000 UTC    True        14.0       False   

       loan_principal_amount  maximum_repayment_amount  eth_price  \
2835            1.281000e+00              1.290476e-18    1883.01   
10030           1.700000e-09              1.730000e-27    1894.89   

          usd_value      interest                                      lender  \
2835   2.412136e+03  9.475890e-03  0x2ae57B80DF1Ea04E5638E780ecc6c96232be65D5   
10030  1.700000e-09  3.000000e-11  0x226d3154Ea70DCC8b983f4BD4061D8BED318E03f   

                                         borrower  \
2835   0x331739Ad658F76dAbc0F5790d42599Ac38DaAD91   
10030  0x331739Ad658F76dAbc0F5790d42599Ac38DaAD91   

                          nft_collateral_contract  nft_collateral_id  active  \
2835   0xacf63e56fd08970b43401492a02f6f38b6635c91               4696   False   
10030  0xacf63e56fd08970b43401492a02f6f38b6635c91               4695   False   

             apr                      loan_erc20denomination  \
2835   90.000000  0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2   
10030  46.008403  0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48   

                     loan_repaid_time loan_liquidation_time loan_no_cleaned  
2835   2023-04-12 13:40:11.000000 UTC                   NaN           32241  
10030  2023-05-06 19:03:23.000000 UTC                   NaN           34175

In [16]:
display(april_mq.head(3).sort_values('loan_id_cleaned', ascending=False))
display(april_nftfi.head(3).sort_values('loan_no_cleaned', ascending=False))

transaction_hash  \
28  0xecaf8a9e276b227506ebc8acc837792370ec83d52e39...   
82  0x2d07d7afcb6324bfc6239c6bcc9bca658a49cd2b227c...   
2   0x0f3522aa06eb28b8cceb590ea80cbd335ab64010dab8...   

                   block_timestamp  loan_id  \
28  2023-04-28 00:40:35.000000 UTC    34311   
82  2023-04-27 13:53:47.000000 UTC    34265   
2   2023-04-02 09:27:59.000000 UTC    31180   

                                    to_address  \
28  0xefa755b82218dc287ed37bf6463c144ae6609076   
82  0xa35b95b60af5205dbd02571d8d72ef83f79a28af   
2   0xf7b18e107eb36797f4ce36de756630b9c30969ad   

                                  from_address  principal_amount  \
28  0x08f6db921b10ce9f7ec4166b9be9dc01a30c34cc           10000.0   
82  0x1797b4235473fbe0e7e44322f01c1b5618ebda41               6.0   
2   0x31d75becb2fe9cc60eb4877b44110aa1cdae90a5             800.0   

    repayment_amount                               erc20_address erc20_name  \
28      10082.191781  0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48       USDc   
82          6.540000  0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2       wETH   
2         826.301370  0x6b175474e89094c44da98b954eedeac495271d0f        DAI   

                          due_date  duration_in_days   apr token_id  \
28  2023-05-28 00:40:35.000000 UTC              30.0  0.10     6096   
82  2024-04-26 13:53:47.000000 UTC             365.0  0.09     9470   
2   2023-06-01 09:27:59.000000 UTC              60.0  0.20     2118   

                            collection_address protocol  amt_in_usd  \
28  0x059edd72cd353df5106d2b9cc5ab83a52287ac3a    nftfi  10000.0000   
82  0x059edd72cd353df5106d2b9cc5ab83a52287ac3a    nftfi  11261.0808   
2   0x026224a2940bfe258d0dbe947919b62fe321f042    nftfi    800.0000   

    roll_over  block_number p2p_p2pool loan_id_cleaned  
28      False      17141128        p2p           34311  
82      False      17137936        p2p           34265  
2       False      16960517        p2p           31180

date blockchain collection_name  \
18  2023-04-05 12:21:23.000000 UTC   ethereum      Art Blocks   
17  2023-04-05 12:13:11.000000 UTC   ethereum      Art Blocks   
16  2023-04-05 05:48:35.000000 UTC   ethereum      Art Blocks   

                  loan_no                 loan_start_time  \
18  v2-1.loan.fixed-31524  2023-04-05 12:21:23.000000 UTC   
17  v2-1.loan.fixed-31522  2023-04-05 12:13:11.000000 UTC   
16  v2-1.loan.fixed-31479  2023-04-05 05:48:35.000000 UTC   

                     loan_due_time  repaid  no_of_days  liquidated  \
18  2024-04-04 12:21:23.000000 UTC   False       365.0       False   
17  2024-04-04 12:13:11.000000 UTC   False       365.0       False   
16  2024-04-04 05:48:35.000000 UTC   False       365.0       False   

    loan_principal_amount  maximum_repayment_amount  eth_price  usd_value  \
18                    6.0              6.540000e-18    1910.54   11463.24   
17                    6.0              6.540000e-18    1912.63   11475.78   
16                    6.0              6.540000e-18    1911.87   11471.22   

    interest                                      lender  \
18      0.54  0x9eff536C3f8fBA72261C687154E9Dc4289b0745D   
17      0.54  0x9eff536C3f8fBA72261C687154E9Dc4289b0745D   
16      0.54  0x9eff536C3f8fBA72261C687154E9Dc4289b0745D   

                                      borrower  \
18  0xfe285aE8A0F7FE2Dd3743966F2f1B85596f63564   
17  0xeFFfDc05e7c5B305Fbd504366B01f2d6424cB8c4   
16  0x3b7F246d8340DE4Fe0495Bfd243FBEa798503C7f   

                       nft_collateral_contract  nft_collateral_id  active  \
18  0x059edd72cd353df5106d2b9cc5ab83a52287ac3a               9408    True   
17  0x059edd72cd353df5106d2b9cc5ab83a52287ac3a               2244    True   
16  0x059edd72cd353df5106d2b9cc5ab83a52287ac3a               9449    True   

    apr                      loan_erc20denomination loan_repaid_time  \
18  9.0  0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2              NaN   
17  9.0  0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2              NaN   
16  9.0  0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2              NaN   

   loan_liquidation_time loan_no_cleaned  
18                   NaN           31524  
17                   NaN           31522  
16                   NaN           31479

In [17]:
# Determine the set of unique IDs present in MQ and not in NFTfi
len(set(list(april_mq.loan_id_cleaned)) - set(list(april_nftfi.loan_no_cleaned)))

402

In [18]:
april_mq.shape, april_nftfi.shape

((1852, 20), (2168, 24))

In [19]:
april_nftfi.shape[0] - april_mq.shape[0]

316

In [20]:
mq_loans_not_in_nftfi = list(
        set(list(april_mq.loan_id_cleaned)) - set(list(april_nftfi.loan_no_cleaned))
    )
mq_loans_not_in_nftfi[:5]

['34059', '31704', '32175', '34463', '31771']

In [21]:
len(mq_loans_not_in_nftfi)

402

In [33]:
# MQ loans not in NFTfi
mq_not_in_nftfi = april_mq[april_mq['loan_id_cleaned'].isin(mq_loans_not_in_nftfi)].reset_index(drop=True)
mq_not_in_nftfi.head(3)

transaction_hash  \
0  0x0f3522aa06eb28b8cceb590ea80cbd335ab64010dab8...   
1  0x18fb1609bf5abff9e1fb2b9f44c1cd3ac1eebc9d1555...   
2  0x7a60e16d71c9b4042d425eb67a950a58f3b8aeb8d759...   

                  block_timestamp  loan_id  \
0  2023-04-02 09:27:59.000000 UTC    31180   
1  2023-04-13 01:40:35.000000 UTC    32542   
2  2023-04-26 10:16:23.000000 UTC    34061   

                                   to_address  \
0  0xf7b18e107eb36797f4ce36de756630b9c30969ad   
1  0x905f48cbaaaf881dbee913ce040c3b26d3bbc6d9   
2  0x376bc7ddbc6a10f32b7421287f1dac6de0ad9f12   

                                 from_address  principal_amount  \
0  0x31d75becb2fe9cc60eb4877b44110aa1cdae90a5             800.0   
1  0x5b20dce335d131c6de0cc11b481ace2aade788df              12.5   
2  0x66ea2d335291a5afa8d3522eef6adfc3f8746e16           12000.0   

   repayment_amount                               erc20_address erc20_name  \
0        826.301370  0x6b175474e89094c44da98b954eedeac495271d0f        DAI   
1         13.085308  0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2       wETH   
2      12295.890411  0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48       USDc   

                         due_date  duration_in_days     apr token_id  \
0  2023-06-01 09:27:59.000000 UTC              60.0  0.2000     2118   
1  2023-07-12 01:40:35.000000 UTC              90.0  0.1899     2949   
2  2023-06-25 10:16:23.000000 UTC              60.0  0.1500     8223   

                           collection_address protocol  amt_in_usd  roll_over  \
0  0x026224a2940bfe258d0dbe947919b62fe321f042    nftfi    800.0000      False   
1  0x059edd72cd353df5106d2b9cc5ab83a52287ac3a    nftfi  23874.3825      False   
2  0x059edd72cd353df5106d2b9cc5ab83a52287ac3a    nftfi  12000.0000      False   

   block_number p2p_p2pool loan_id_cleaned  
0      16960517        p2p           31180  
1      17035683        p2p           32542  
2      17129749        p2p           34061

In [23]:
# We need to explore this table
mq_not_in_nftfi.erc20_name.unique()

array(['DAI', 'wETH', 'USDc'], dtype=object)

## NFTfi loans not in MQ

In [24]:
nftfi_loans_not_in_mq = list(
        set(list(april_nftfi.loan_no_cleaned)) - set(list(april_mq.loan_id_cleaned))
    )
len(nftfi_loans_not_in_mq)

716

In [25]:
## LOANS IN NFTFI AND NOT IN MQ
nftfi_not_in_mq = april_nftfi[april_nftfi['loan_no_cleaned'].isin(nftfi_loans_not_in_mq)].reset_index(drop=True)
nftfi_not_in_mq

date blockchain  \
0    2023-04-11 20:14:59.000000 UTC   ethereum   
1    2023-04-11 20:24:11.000000 UTC   ethereum   
2    2023-04-12 21:55:47.000000 UTC   ethereum   
3    2023-04-12 21:56:23.000000 UTC   ethereum   
4    2023-04-12 21:57:11.000000 UTC   ethereum   
..                              ...        ...   
713  2023-04-25 21:25:35.000000 UTC   ethereum   
714  2023-04-25 21:37:11.000000 UTC   ethereum   
715  2023-04-03 22:17:23.000000 UTC   ethereum   
716  2023-04-22 14:24:59.000000 UTC   ethereum   
717  2023-04-29 12:46:47.000000 UTC   ethereum   

                          collection_name                         loan_no  \
0    Friendship Bracelets by Alexis André           v2-1.loan.fixed-32381   
1    Friendship Bracelets by Alexis André           v2-1.loan.fixed-32384   
2    Friendship Bracelets by Alexis André           v2-1.loan.fixed-32532   
3    Friendship Bracelets by Alexis André           v2-1.loan.fixed-32533   
4    Friendship Bracelets by Alexis André           v2-1.loan.fixed-32535   
..                                    ...                             ...   
713                               NFTrees  v2.loan.fixed.collection-33989   
714                               NFTrees           v2-1.loan.fixed-33990   
715                                 Azuki  v2.loan.fixed.collection-31340   
716                                 Azuki  v2.loan.fixed.collection-33603   
717                                 Azuki  v2.loan.fixed.collection-34479   

                    loan_start_time                   loan_due_time  repaid  \
0    2023-04-11 20:14:59.000000 UTC  2023-05-15 20:14:59.000000 UTC   False   
1    2023-04-11 20:24:11.000000 UTC  2023-05-16 20:24:11.000000 UTC   False   
2    2023-04-12 21:55:47.000000 UTC  2023-06-03 21:55:47.000000 UTC   False   
3    2023-04-12 21:56:23.000000 UTC  2023-05-19 21:56:23.000000 UTC   False   
4    2023-04-12 21:57:11.000000 UTC  2023-05-19 21:57:11.000000 UTC   False   
..                              ...                             ...     ...   
713  2023-04-25 21:25:35.000000 UTC  2023-05-16 21:25:35.000000 UTC    True   
714  2023-04-25 21:37:11.000000 UTC  2023-05-16 21:37:11.000000 UTC    True   
715  2023-04-03 22:17:23.000000 UTC  2023-04-24 22:17:23.000000 UTC    True   
716  2023-04-22 14:24:59.000000 UTC  2023-05-13 14:24:59.000000 UTC    True   
717  2023-04-29 12:46:47.000000 UTC  2023-05-20 12:46:47.000000 UTC   False   

     no_of_days  liquidated  loan_principal_amount  maximum_repayment_amount  \
0          34.0       False           1.654300e-01              1.760630e-19   
1          35.0       False           1.654320e-01              1.763780e-19   
2          52.0       False           1.546560e-01              1.698590e-19   
3          37.0       False           1.565430e-01              1.674920e-19   
4          37.0       False           1.564340e-01              1.673760e-19   
..          ...         ...                    ...                       ...   
713        21.0       False           7.000000e-09              7.070479e-27   
714        21.0       False           7.000000e-09              7.070479e-27   
715        21.0       False           1.400000e+04              1.425775e-14   
716        21.0       False           1.650000e+04              1.677530e-14   
717        21.0       False           1.190000e+04              1.191027e-14   

     eth_price     usd_value      interest  \
0      1893.88  3.133046e+02  1.063300e-02   
1      1888.23  3.123737e+02  1.094600e-02   
2      1917.11  2.964926e+02  1.520300e-02   
3      1917.11  3.001102e+02  1.094900e-02   
4      1917.11  2.999012e+02  1.094200e-02   
..         ...           ...           ...   
713    1870.25  7.000000e-09  7.047945e-11   
714    1868.17  7.000000e-09  7.047945e-11   
715    1813.83  1.400000e+04  2.577534e+02   
716    1854.59  1.650000e+04  2.753014e+02   
717    1897.14  1.190000e+04  1.026986e+01   

                                         l

In [26]:
# Explore the resulting set of nftfi_not_in_mq. Is there any pattern?
nftfi_not_in_mq.describe()

no_of_days  loan_principal_amount  maximum_repayment_amount  \
count  718.000000           7.180000e+02              7.180000e+02   
mean    13.424791           1.556868e+02              1.578288e-16   
std     12.125416           1.127875e+03              1.142259e-15   
min      1.000000           5.000000e-10              5.140000e-28   
25%      7.000000           4.100000e-01              4.200000e-19   
50%      7.000000           6.900000e-01              7.053535e-19   
75%     14.000000           2.540000e+00              2.612568e-18   
max    180.000000           1.650000e+04              1.677530e-14   

         eth_price     usd_value      interest  nft_collateral_id         apr  
count   718.000000  7.180000e+02  7.180000e+02       7.180000e+02  718.000000  
mean   1943.408788  5.413593e+03  2.142018e+00       1.946874e+09   59.619488  
std     107.135581  2.061052e+04  1.763550e+01       1.797736e+10   41.626792  
min    1778.230000  5.000000e-10  1.400000e-11       1.100000e+01    0.973333  
25%    1863.770000  7.709757e+02  5.801432e-03       2.120000e+03   30.008722  
50%    1904.050000  1.303909e+03  1.258800e-02       4.633000e+03   51.237002  
75%    2088.920000  4.633631e+03  5.050219e-02       8.174500e+03   79.995194  
max    2124.200000  3.600285e+05  2.753014e+02       1.710001e+11  299.000000

### Are discrepancies due to loan type which is not accounted for?

In [39]:
# How about loan numbers, how do they vary from a naming standpoint?
# x[0]+x[1] if len(x) >2 else x[0] is such that it returns "v2-1.loan.fixed" from "v2-1.loan.fixed-32381", 
#    else returns "v2.loan.fixed.collection" from "v2.loan.fixed.collection-34479"

nftfi_not_in_mq.loan_no.astype(str).str.split('-').apply(lambda x: x[0]+'-'+x[1] if len(x) >2 else x[0]).unique()
# it seems from the above result that [v2-1.loan.fixed', 'v2.loan.fixed.collection'] are the two loan types which are not accounted for

array(['v2-1.loan.fixed', 'v2.loan.fixed.collection'], dtype=object)

In [40]:
# Is the above true as well for the whole table set?
nftfi_loans_not_in_mq = list(
        set(list(nftfi.loan_no_cleaned)) - set(list(mq.loan_id_cleaned))
    )
print(len(nftfi_loans_not_in_mq))
# How about loan numbers, how do they vary from a naming standpoint?
# x[0]+x[1] if len(x) >2 else x[0] is such that it returns "v2-1.loan.fixed" from "v2-1.loan.fixed-32381", 
#    else returns "v2.loan.fixed.collection" from "v2.loan.fixed.collection-34479"

nftfi_not_in_mq.loan_no.astype(str).str.split('-').apply(lambda x: x[0]+'-'+x[1] if len(x) >2 else x[0]).unique()
# it seems from the above result that [v2-1.loan.fixed', 'v2.loan.fixed.collection'] are the two loan types which are not accounted for

4974


array(['v2-1.loan.fixed', 'v2.loan.fixed.collection'], dtype=object)

### Is it due to missed erc20 token denomination?

In [29]:
# Check if it can be due to missing token
usdc_address = '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48'  # https://etherscan.io/token/0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48
usdt_address = '0xdAC17F958D2ee523a2206206994597C13D831ec7'  # https://etherscan.io/token/0xdac17f958d2ee523a2206206994597c13d831ec7
dai_address = '0x6B175474E89094C44Da98b954EedeAC495271d0F'  # https://etherscan.io/token/0x6b175474e89094c44da98b954eedeac495271d0f
weth_address = '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2'

stablecoin_addresses = [usdc_address, usdt_address, dai_address, weth_address]

stablecoin_names = ['usdc', 'usdt', 'dai', 'weth']
for stablecoin_address, stablecoin_name in zip(stablecoin_addresses, stablecoin_names):
    stablecoin_address = stablecoin_address.lower()
    nftfi_not_in_mq.loc[nftfi_not_in_mq['loan_erc20denomination'] == stablecoin_address, 'loan_erc20denomination_name'] = stablecoin_name

In [30]:
nftfi_not_in_mq.loan_erc20denomination_name.unique()
# It does not seem to be linked to missing token.

array(['weth', 'dai', 'usdc'], dtype=object)

In [36]:
nftfi_loans_not_in_mq = list(
        set(list(nftfi.loan_no_cleaned)) - set(list(mq.loan_id_cleaned))
    )
print(len(nftfi_loans_not_in_mq))
# How about loan numbers, how do they vary from a naming standpoint?
# x[0]+x[1] if len(x) >2 else x[0] is such that it returns "v2-1.loan.fixed" from "v2-1.loan.fixed-32381", 
#    else returns "v2.loan.fixed.collection" from "v2.loan.fixed.collection-34479"

nftfi_not_in_mq.loan_no.astype(str).str.split('-').apply(lambda x: x[0]+'-'+x[1] if len(x) >2 else x[0]).unique()
# it seems from the above result that [v2-1.loan.fixed', 'v2.loan.fixed.collection'] are the two loan types which are not accounted for

4974


array(['v2-1.loan.fixed', 'v2.loan.fixed.collection'], dtype=object)

NameError: name 'aa' is not defined

In [ ]:
april_nftfi.blockchain.unique()

In [ ]:
import numpy as np
april_nftfi[april_nftfi['blockchain'] == np.nan]

In [ ]:
# TODO:
# spreadsheet of different transactions that are missing, or are more of them on MQ dataset. we need clear doc discussing which tsx are missing, which are added,
# what are the volume differences coming from.

# try and check that the eth value of those loans per transaction is the same.
# check if volumes per loanerc20 collateral are matching